**Glove Embeddings== 300 Features Extracted**

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import logging
import pandas as pd
import numpy as np 
import gensim
import nltk
nltk.download('stopwords') 
nltk.download('punkt')
import re
from bs4 import BeautifulSoup
import csv
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS =nltk.corpus.stopwords.words('english')

def clean_text(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split()if word not in STOPWORDS) # delete stopwords from text
    return text

In [5]:
#REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
#BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
#STOPWORDS =nltk.corpus.stopwords.words('english')

In [6]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/glove.6B.300d.txt" 
wv = pd.read_table(path, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [7]:
def vec(w):
  return wv.loc[w].as_matrix()

In [8]:
def w2v_tokenize_text(text):
  tokens = []
  for sent in nltk.sent_tokenize(text, language='english'):
    for word in nltk.word_tokenize(sent, language='english'):
      if len(word) < 2:
        continue
      tokens.append(word)
  return tokens

In [9]:
def wordprint(words,wv):
    all_words, mean = set(), []
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.index:
           a=np.array(wv.loc[word])
           mean.append(a)
    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(300,)   
    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [10]:
def  word_averaging_listn(wv, text_list):
  return np.vstack([wordprint(post,wv) for post in text_list ])

In [11]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [12]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [13]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [14]:
#define bins
bins = [0,2,3,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [15]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [16]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [17]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [18]:
df.iloc[:,0] = df.iloc[:,0].apply(clean_text)
print(df.iloc[:100,0]) 

0     constants alpha beta frac x alpha x+ beta frac...
1     length segment points 2a a4 4 1 2 sqrt 10 unit...
2     chewbacca 20 pieces cherry gum 30 pieces grape...
3     constants b let f x left begin array cl ax + b...
4             calculate sum 1 + 3 + 5 + cdots + 15 + 17
                            ...                        
95    largest value x satisfies equation sqrt 2x 4x ...
96    sum left dfrac 1 3 right + left dfrac 1 3 righ...
97    map 12centimeter length represents 72 kilomete...
98                      find sum values x 2 x23x2 4 x 4
99              given 0 f g 8 f x x2 + 8 g x x2 4 value
Name: Problem , Length: 100, dtype: object


In [19]:
comtdata=df

In [20]:
test_tokenized = comtdata.apply(lambda r: w2v_tokenize_text(r.iloc[0]), axis=1).values

In [21]:
df_new=pd.DataFrame(test_tokenized)

In [22]:
df_new.head()

,0
0,"[constants, alpha, beta, frac, alpha, x+, beta..."
1,"[length, segment, points, 2a, a4, sqrt, 10, un..."
2,"[chewbacca, 20, pieces, cherry, gum, 30, piece..."
3,"[constants, let, left, begin, array, cl, ax, t..."
4,"[calculate, sum, cdots, 15, 17]"


In [23]:
X_comtdata_averagen=word_averaging_listn(wv, test_tokenized)

In [24]:
X_comtdata_averagen.shape 

(1744, 300)

In [25]:
x=X_comtdata_averagen 

In [26]:
X = pd.DataFrame(x)
X

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.002601,0.069051,0.024335,-0.033869,-0.040151,-0.018430,-0.014827,-0.008229,0.034968,-0.089211,...,0.008067,-0.017670,-0.058953,-0.076793,0.003568,0.011851,0.038819,-0.025364,0.050370,0.045423
1,-0.116452,0.113243,0.021199,-0.029057,-0.100692,0.030531,-0.016861,0.047032,0.086258,-0.335942,...,0.033680,-0.023741,-0.012486,-0.016859,0.085727,0.084629,-0.016294,-0.088710,0.007819,-0.016146
2,-0.114690,0.068149,-0.052366,-0.003492,-0.057075,0.029782,-0.019974,-0.022189,0.036161,-0.260943,...,0.007772,-0.045431,0.009296,0.010240,-0.010566,-0.025869,-0.002291,0.001537,-0.002885,-0.056085
3,-0.060044,0.001034,-0.016318,-0.018206,-0.062817,0.059124,-0.008970,0.004282,0.028531,-0.404915,...,-0.018972,0.031668,-0.001115,-0.037229,0.036026,0.020596,0.075927,-0.098333,0.037183,-0.037534
4,-0.109358,0.033520,0.064459,0.011835,-0.104055,0.008890,0.101534,-0.071097,-0.014442,-0.216769,...,0.049903,-0.041085,0.055216,-0.088967,0.089463,0.015155,-0.008312,-0.113034,-0.033500,-0.076046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1739,-0.038660,0.064017,0.037520,0.035148,-0.126032,-0.036731,0.049624,-0.012202,-0.004470,-0.265016,...,0.028917,0.064441,0.012782,0.042455,0.034327,0.037906,0.033499,-0.084711,0.071507,-0.013839
1740,-0.035832,0.011162,0.021095,0.011592,-0.076143,0.061837,0.012090,-0.034667,0.019889,-0.157855,...,0.021873,-0.108254,-0.025975,-0.104304,0.031799,0.134799,-0.038208,-0.153331,-0.028206,-0.007938
1741,-0.009391,-0.051511,0.046161,-0.037776,-0.006842,0.056023,-0.066021,-0.047209,0.051672,-0.045083,...,0.105926,0.079233,0.025306,-0.015541,0.017567,-0.029685,0.056411,0.096892,0.119761,0.007673
1742,-0.072737,0.098112,0.064755,-0.008766,0.004100,0.079810,-0.069718,-0.038938,-0.067618,-0.237141,...,-0.028283,0.124893,-0.070227,0.009369,0.031895,-0.013830,0.030191,-0.092734,-0.039990,-0.015957


In [27]:
from google.colab import files
X.to_csv('glove.csv') 
files.download('glove.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [28]:
path = "/content/drive/MyDrive/newpro/threeclass_problem/glove.csv"
X = pd.read_csv(path)
X.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
0,0,0.002601,0.069051,0.024335,-0.033869,-0.040151,-0.018430,-0.014827,-0.008229,0.034968,...,0.008067,-0.017670,-0.058953,-0.076793,0.003568,0.011851,0.038819,-0.025364,0.050370,0.045423
1,1,-0.116452,0.113243,0.021199,-0.029057,-0.100692,0.030531,-0.016861,0.047032,0.086258,...,0.033680,-0.023741,-0.012486,-0.016859,0.085727,0.084629,-0.016294,-0.088710,0.007819,-0.016146
2,2,-0.114690,0.068149,-0.052366,-0.003492,-0.057075,0.029782,-0.019974,-0.022189,0.036161,...,0.007772,-0.045431,0.009296,0.010240,-0.010566,-0.025869,-0.002291,0.001537,-0.002885,-0.056085
3,3,-0.060044,0.001034,-0.016318,-0.018206,-0.062817,0.059124,-0.008970,0.004282,0.028531,...,-0.018972,0.031668,-0.001115,-0.037229,0.036026,0.020596,0.075927,-0.098333,0.037183,-0.037534
4,4,-0.109358,0.033520,0.064459,0.011835,-0.104055,0.008890,0.101534,-0.071097,-0.014442,...,0.049903,-0.041085,0.055216,-0.088967,0.089463,0.015155,-0.008312,-0.113034,-0.033500,-0.076046


In [32]:
X.drop(X.columns[[0]], axis=1, inplace=True)

In [33]:
y=df['Level ']

In [34]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 300), (524, 300))

In [35]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

3    48.196721
1    29.590164
2    22.213115
Name: Level , dtype: float64


3    46.946565
1    29.961832
2    23.091603
Name: Level , dtype: float64


In [36]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [37]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test)

Perceptron F1 score: 0.5433125326827128


**Decision Tree**

In [38]:
clf_dt = DecisionTreeClassifier(max_depth=5)

In [39]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=5)

In [40]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [41]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [42]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [43]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994


**Random Forest**

In [44]:
clf_rf = RandomForestClassifier()

In [45]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier()

In [46]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [47]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9547    0.9917    0.9728       361
           2     0.9961    0.9446    0.9697       271
           3     0.9932    0.9932    0.9932       588

    accuracy                         0.9820      1220
   macro avg     0.9813    0.9765    0.9786      1220
weighted avg     0.9824    0.9820    0.9819      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5782    0.5414    0.5592       157
           2     0.3333    0.0413    0.0735       121
           3     0.5718    0.8415    0.6809       246

    accuracy                         0.5668       524
   macro avg     0.4945    0.4747    0.4379       524
weighted avg     0.5187    0.5668    0.5042       524



In [48]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994
1,RandomForest,0.981967,0.981967,0.982443,0.981949,0.566794,0.566794,0.518672,0.504198


**AdaBoost**

In [49]:
clf_adaboost =  AdaBoostClassifier()

In [50]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [51]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [52]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6715    0.6399    0.6553       361
           2     0.5672    0.4207    0.4831       271
           3     0.7096    0.8146    0.7585       588

    accuracy                         0.6754      1220
   macro avg     0.6494    0.6251    0.6323      1220
weighted avg     0.6667    0.6754    0.6668      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4832    0.4586    0.4706       157
           2     0.2143    0.1488    0.1756       121
           3     0.5533    0.6545    0.5996       246

    accuracy                         0.4790       524
   macro avg     0.4169    0.4206    0.4153       524
weighted avg     0.4540    0.4790    0.4631       524



In [53]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994
1,RandomForest,0.981967,0.981967,0.982443,0.981949,0.566794,0.566794,0.518672,0.504198
2,Adaboost,0.675410,0.675410,0.666704,0.666788,0.479008,0.479008,0.454003,0.463053


**Gradient Boosting Classifier**

In [54]:
clf_gbm = GradientBoostingClassifier()

In [55]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [56]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [57]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9455    0.9612    0.9533       361
           2     0.9830    0.8524    0.9130       271
           3     0.9434    0.9915    0.9668       588

    accuracy                         0.9516      1220
   macro avg     0.9573    0.9350    0.9444      1220
weighted avg     0.9528    0.9516    0.9509      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5205    0.5669    0.5427       157
           2     0.3333    0.1157    0.1718       121
           3     0.6013    0.7602    0.6715       246

    accuracy                         0.5534       524
   macro avg     0.4850    0.4809    0.4620       524
weighted avg     0.5152    0.5534    0.5175       524



In [58]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994
1,RandomForest,0.981967,0.981967,0.982443,0.981949,0.566794,0.566794,0.518672,0.504198
2,Adaboost,0.675410,0.675410,0.666704,0.666788,0.479008,0.479008,0.454003,0.463053
3,GBM,0.951639,0.951639,0.952798,0.950879,0.553435,0.553435,0.515197,0.517489


**SVM Model**

In [59]:
linear_svm = SVC(kernel='linear', C=1)

In [60]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, kernel='linear')

In [61]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [62]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5994    0.5762    0.5876       361
           2     0.6667    0.0148    0.0289       271
           3     0.6101    0.8997    0.7271       588

    accuracy                         0.6074      1220
   macro avg     0.6254    0.4969    0.4479      1220
weighted avg     0.6195    0.6074    0.5307      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.5643    0.5032    0.5320       157
           2     0.0000    0.0000    0.0000       121
           3     0.5561    0.8659    0.6773       246

    accuracy                         0.5573       524
   macro avg     0.3735    0.4563    0.4031       524
weighted avg     0.4302    0.5573    0.4773       524



In [63]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994
1,RandomForest,0.981967,0.981967,0.982443,0.981949,0.566794,0.566794,0.518672,0.504198
2,Adaboost,0.675410,0.675410,0.666704,0.666788,0.479008,0.479008,0.454003,0.463053
3,GBM,0.951639,0.951639,0.952798,0.950879,0.553435,0.553435,0.515197,0.517489
4,SVM,0.607377,0.607377,0.619530,0.530740,0.557252,0.557252,0.430157,0.477346


**KNN**

In [64]:
 knn = KNeighborsClassifier(n_neighbors=7)

In [65]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [66]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [67]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5624    0.7119    0.6284       361
           2     0.5103    0.2731    0.3558       271
           3     0.7217    0.7585    0.7396       588

    accuracy                         0.6369      1220
   macro avg     0.5981    0.5812    0.5746      1220
weighted avg     0.6276    0.6369    0.6214      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4634    0.6051    0.5249       157
           2     0.3390    0.1653    0.2222       121
           3     0.6115    0.6463    0.6285       246

    accuracy                         0.5229       524
   macro avg     0.4713    0.4722    0.4585       524
weighted avg     0.5042    0.5229    0.5036       524



In [68]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994
1,RandomForest,0.981967,0.981967,0.982443,0.981949,0.566794,0.566794,0.518672,0.504198
2,Adaboost,0.675410,0.675410,0.666704,0.666788,0.479008,0.479008,0.454003,0.463053
3,GBM,0.951639,0.951639,0.952798,0.950879,0.553435,0.553435,0.515197,0.517489
4,SVM,0.607377,0.607377,0.619530,0.530740,0.557252,0.557252,0.430157,0.477346
5,KNN,0.636885,0.636885,0.627595,0.621441,0.522901,0.522901,0.504220,0.503613


**GNB**

In [69]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [70]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [71]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [72]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5023    0.6177    0.5540       361
           2     0.4510    0.2546    0.3255       271
           3     0.6549    0.6939    0.6738       588

    accuracy                         0.5738      1220
   macro avg     0.5360    0.5221    0.5178      1220
weighted avg     0.5644    0.5738    0.5610      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4670    0.5860    0.5198       157
           2     0.2537    0.1405    0.1809       121
           3     0.6192    0.6545    0.6364       246

    accuracy                         0.5153       524
   macro avg     0.4467    0.4603    0.4457       524
weighted avg     0.4892    0.5153    0.4962       524



In [73]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.678689,0.678689,0.692258,0.648282,0.505725,0.505725,0.436746,0.457994
1,RandomForest,0.981967,0.981967,0.982443,0.981949,0.566794,0.566794,0.518672,0.504198
2,Adaboost,0.675410,0.675410,0.666704,0.666788,0.479008,0.479008,0.454003,0.463053
3,GBM,0.951639,0.951639,0.952798,0.950879,0.553435,0.553435,0.515197,0.517489
4,SVM,0.607377,0.607377,0.619530,0.530740,0.557252,0.557252,0.430157,0.477346
5,KNN,0.636885,0.636885,0.627595,0.621441,0.522901,0.522901,0.504220,0.503613
6,GNB,0.573770,0.573770,0.564432,0.560999,0.515267,0.515267,0.489221,0.496246
